<a href="https://colab.research.google.com/github/abhishek203/E-Abhishek/blob/master/Project%20itsp%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import os
import io
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img
import matplotlib.pyplot as plt

In [0]:
#dowloading the file from github
path_to_zip = tf.keras.utils.get_file(
    'facial_expressions-master.zip', origin='https://codeload.github.com/muxspace/facial_expressions/zip/master',
    extract=True)

path_to_csvfile = os.path.dirname(path_to_zip)+"/facial_expressions-master/data/legend.csv"
path_to_images = os.path.dirname(path_to_zip) + "/facial_expressions-master/images"

In [0]:
data_dir = os.path.dirname(path_to_zip) + "/facial_expressions-master/images"

In [50]:
file = pd.read_csv(path_to_csvfile)
legend = file.sort_values(by=['image'])
legend.replace({'emotion':str.lower} )

,user.id,image,emotion
146,302,AJ_Cook_0001.jpg,happiness
147,302,AJ_Lamas_0001.jpg,happiness
4,dwdii,Aaron_Eckhart_0001.jpg,neutral
5,302,Aaron_Guiel_0001.jpg,happiness
6,302,Aaron_Patterson_0001.jpg,neutral
...,...,...,...
12768,dwdii,Zydrunas_Ilgauskas_0001.jpg,neutral
3,628,facial-expressions_2868582k.jpg,fear
2,628,facial-expressions_2868584k.jpg,disgust
1,628,facial-expressions_2868585k.jpg,surprise


In [0]:
no_of_entries = (13690*8)//10
batch_size = 32
IMG_HEIGHT = 300
IMG_WIDTH = 300
epochs = 15
matrix = legend.to_numpy()#in matrix format

In [0]:
image=[]
emotions=[]
for i in range(no_of_entries):
  emotions.append(matrix[i][2])
  emotions[i].lower()
  path=path_to_images+"/"+matrix[i][1]
  img=load_img(path,color_mode="grayscale",target_size=(300,300))
  img1 = tf.keras.preprocessing.image.img_to_array(img)
  img2 = tf.convert_to_tensor(img1)
  img2 = tf.image.convert_image_dtype(img2, tf.float32)
  img2 = img2/255
  image.append(img2)

In [53]:
len(image)

10952

In [0]:
emotion_set = set(emotions)

In [55]:
emotion_set

{'ANGER',
 'DISGUST',
 'FEAR',
 'HAPPINESS',
 'NEUTRAL',
 'SADNESS',
 'SURPRISE',
 'anger',
 'contempt',
 'disgust',
 'fear',
 'happiness',
 'neutral',
 'sadness',
 'surprise'}

In [0]:
emotion_vec = []
for i in range(no_of_entries):
  if emotions[i] == 'ANGER' or emotions[i]=='anger':
    emotion_vec.append(0)
  if emotions[i] == 'NEUTRAL' or emotions[i]=='neutral':
    emotion_vec.append(1)
  if emotions[i] == 'CONTEMPT' or emotions[i]=='contempt':
    emotion_vec.append(2)
  if emotions[i] == 'DISGUST' or emotions[i]=='disgust':
    emotion_vec.append(3)
  if emotions[i] == 'FEAR' or emotions[i]=='fear':
    emotion_vec.append(4)
  if emotions[i] == 'HAPPINESS' or emotions[i]=='happiness':
    emotion_vec.append(5)
  if emotions[i] == 'SADNESS' or emotions[i]=='sadness':
    emotion_vec.append(6)
  if emotions[i] == 'SURPRISE' or emotions[i]=='surprise':
    emotion_vec.append(7)  

In [57]:
emotion_vec[100]

5

In [58]:
emotions[100]

'happiness'

In [0]:
emotion_vec = tf.keras.utils.to_categorical(emotion_vec)
emotion_vec = tf.convert_to_tensor(emotion_vec)

In [60]:
emotion_vec[100]

<tf.Tensor: shape=(8,), dtype=float32, numpy=array([0., 0., 0., 0., 0., 1., 0., 0.], dtype=float32)>

In [0]:
dataset_image = tf.data.Dataset.from_tensor_slices((image,emotion_vec))
dataset_image = dataset_image.batch(batch_size, drop_remainder=True)

In [0]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,1)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(8,activation = 'softmax')
])

In [0]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [64]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 300, 300, 16)      160       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 150, 150, 16)      0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 150, 150, 32)      4640      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 75, 75, 64)        18496     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 37, 37, 64)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 87616)            

In [66]:
model.fit(dataset_image,batch_size=batch_size,epochs=10)

Epoch 1/10
342/342 [==============================] - 14s 41ms/step - loss: 1.0800 - accuracy: 0.5001
Epoch 2/10
342/342 [==============================] - 14s 41ms/step - loss: 1.0921 - accuracy: 0.4850
Epoch 3/10
342/342 [==============================] - 14s 40ms/step - loss: 1.0857 - accuracy: 0.4879
Epoch 4/10
342/342 [==============================] - 14s 41ms/step - loss: 1.0615 - accuracy: 0.5237
Epoch 5/10
342/342 [==============================] - 14s 40ms/step - loss: 0.9395 - accuracy: 0.5993
Epoch 6/10
342/342 [==============================] - 14s 41ms/step - loss: 0.7363 - accuracy: 0.7309
Epoch 7/10
342/342 [==============================] - 14s 40ms/step - loss: 0.6452 - accuracy: 0.7630
Epoch 8/10
342/342 [==============================] - 14s 41ms/step - loss: 0.5248 - accuracy: 0.8074
Epoch 9/10
342/342 [==============================] - 14s 40ms/step - loss: 0.4218 - accuracy: 0.8468
Epoch 10/10
342/342 [==============================] - 14s 41ms/step - loss: 0.303